In [ ]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.io import DataLoader, TableNamesHOSP, TableNamesICU, ParquetConverter
import pandas as pd
import dask.dataframe as dd

In [ ]:
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=4, processes=True, memory_limit='8GB')
client

## Scan the directory

In [ ]:

data_loader = DataLoader()

# Scan the directory
data_loader.scan_mimic_directory()


data_loader.study_tables_info

In [ ]:
%%time

subject_ids = data_loader.get_partial_subject_id_list_for_partial_loading(num_subjects=5)
print(subject_ids)

df = data_loader.load_table(table_name=TableNamesHOSP.DIAGNOSES_ICD, partial_loading=True, subject_ids=subject_ids)
# df = df.compute()
df

In [ ]:
tables_dict = data_loader.load_all_study_tables(partial_loading=True, num_subjects=5)


In [ ]:
print(tables_dict.keys())


patients_df        = tables_dict[TableNamesHOSP.PATIENTS.value]
admissions_df      = tables_dict[TableNamesHOSP.ADMISSIONS.value]
diagnoses_icd_df   = tables_dict[TableNamesHOSP.DIAGNOSES_ICD.value]
poe_df             = tables_dict[TableNamesHOSP.POE.value]
d_icd_diagnoses_df = tables_dict[TableNamesHOSP.D_ICD_DIAGNOSES.value]
poe_detail_df      = tables_dict[TableNamesHOSP.POE_DETAIL.value]

In [ ]:
df12 = patients_df.merge(admissions_df, on='subject_id', how='inner')
df34 = diagnoses_icd_df.merge(d_icd_diagnoses_df, on=('icd_code', 'icd_version'), how='inner')
df56 = poe_df.merge(poe_detail_df, on=('poe_id', 'poe_seq', 'subject_id'), how='left')

df1234   = df12.merge(df34, on=('subject_id', 'hadm_id'), how='inner')
df123456 = df1234.merge(df56, on=('subject_id', 'hadm_id'), how='inner')

In [ ]:
df123456c = df123456.compute()
df123456c

In [ ]:
# Filter age
df123456 = df123456[(df123456.anchor_age >= 18.0) & (df123456.anchor_age <= 75.0)]

# Filter icd_version. Ony keep the ICD 10
df123456 = df123456[df123456.icd_version == 10]

# remove columns discontinued_by_poe_id
df123456 = df123456.drop(columns=['discontinued_by_poe_id'])


In [ ]:
df = df123456.compute()
df.keys()

In [ ]:
# how to check if either one or both of admittime or dischtime are null
admissions_df_full    = data_loader.load_table(table_name=TableNamesHOSP.ADMISSIONS, partial_loading=False)
diagnoses_icd_df_full = data_loader.load_table(table_name=TableNamesHOSP.DIAGNOSES_ICD, partial_loading=False)

# filter out the admissions where patient dies
admissions_df_full = admissions_df_full[admissions_df_full.deathtime.isnull()]

# filter out the diagnoses where icd_version is not 10
diagnoses_icd_df_full = diagnoses_icd_df_full[diagnoses_icd_df_full.icd_version == 10]


dfa = admissions_df_full.compute()
dfd = diagnoses_icd_df_full.compute()

In [ ]:
diagnoses_icd_df_full.shape[0].compute()

In [ ]:
diagnoses_icd_df_full.size.compute()